In [2]:
from qiskit.circuit.library import ExcitationPreserving
from qiskit import Aer, execute

#from pennylane import numpy as np
import numpy as np
import pandas as pd

import qutip

n_wires = 4
n_layers = 2
iteracoes = 3
n_targets = 10


def ExcitationPreserving2(n_layers, n_wires, n_targets):

    circuit = ExcitationPreserving(n_wires, reps=n_layers, insert_barriers=True, entanglement='linear')

    num_pars = len(circuit.parameters)
    values = np.random.rand(num_pars)
    new_circuit = circuit.assign_parameters(values)


    def normaliza(v):
        v = v / (np.linalg.norm(v))    
        return v

    def cria_alvos(n_targets, n_wires):
        target_states = []
        for i in range(n_targets):
            target_states.append(normaliza(np.random.rand(1, 2**(n_wires))))
        return target_states  


    print("Circuito gerado\n: {}".format(new_circuit.decompose().draw()))

    backend = Aer.get_backend('statevector_simulator')
    ket = execute(new_circuit,backend).result().get_statevector()

    target_states = cria_alvos(n_targets, n_wires)

    #print("Estado gerado pelo modelo: {}".format(ket))

    ket = np.array(ket)

    def ket_in_amplt(v):
        aux = []
        for i in range(len(v)):
            v[i] = np.dot(v[i],v[i])
            v[i] = np.linalg.norm(v[i])
            aux.append(float(v[i].real))
        aux = np.array(aux)    
        return aux
    
    amplt = ket_in_amplt(ket)   


    def fidelity(target_states, amplt):
        fidelity_sum = 0
        for target_state in target_states:
            output_probs = amplt
            fidelity = np.vdot(target_state, output_probs) ** 2
            fidelity_sum += fidelity
    
        average_fidelity = fidelity_sum / len(target_states)
        return average_fidelity
    

    def compute_Q_ptrace(ket, N):
        ket = qutip.Qobj(ket, dims=[[2]*(N), [1]*(N)]).unit()
        entanglement_sum = 0
        for k in range(N):
            rho_k_sq = ket.ptrace([k])**2
            entanglement_sum += rho_k_sq.tr()  
   
        Q = 2*(1 - (1/N)*entanglement_sum)
        return Q


    Q = compute_Q_ptrace(ket, n_wires)
    print("\nGrau de Emaranhamento: {}".format(Q))

    E = fidelity(target_states, amplt)
    print("Expressabilidade: {}".format(E))

    #print("\n\nEstado gerado: {}".format(ket))
    #print("\n\n\n\n\n")

    k=[Q,E]
    return k



k=[]
for i in range(iteracoes):
    print("\n=============")
    print("= Modelo: {} =".format(i+1))
    print("=============\n")
    k.append(ExcitationPreserving2(n_layers, n_wires, n_targets))


print("\n\nResultados:\n")
for i in range(iteracoes):
    print(" Modelo {} \n Grau de emaranhamento: {} \n Expressabilidade: {} \n".format(i+1,k[i][0],k[i][1]))


def pand(k):
    eman=[]
    exp=[]

    for i in range(iteracoes):
        eman.append(k[i][0])
    for i in range(iteracoes):
        exp.append(k[i][1]) 

    dic = {"Grau de Emaranhamento": eman, "Expressabilidade":exp}
    df = pd.DataFrame(dic)
    return df

df = pand(k)
df.describe()


= Modelo: 1 =

Circuito gerado
:      ┌───────────────────────┐ ░ ┌─────────────────────────┐»
q_0: ┤ Rz(0.613782461452915) ├─░─┤0                        ├»
     ├───────────────────────┤ ░ │  Rxx(0.102358662581396) │»
q_1: ┤ Rz(0.423571374674465) ├─░─┤1                        ├»
     ├───────────────────────┤ ░ └─────────────────────────┘»
q_2: ┤ Rz(0.581247727916196) ├─░────────────────────────────»
     ├───────────────────────┤ ░                            »
q_3: ┤ Rz(0.813314608346559) ├─░────────────────────────────»
     └───────────────────────┘ ░                            »
«     ┌─────────────────────────┐                           »
«q_0: ┤0                        ├───────────────────────────»
«     │  Ryy(0.102358662581396) │┌─────────────────────────┐»
«q_1: ┤1                        ├┤0                        ├»
«     └─────────────────────────┘│  Rxx(0.379349638908776) │»
«q_2: ───────────────────────────┤1                        ├»
«                                └──

,Grau de Emaranhamento,Expressabilidade
count,3.0,3.000000
mean,0.0,0.055313
std,0.0,0.018109
min,0.0,0.035629
25%,0.0,0.047336
50%,0.0,0.059044
75%,0.0,0.065154
max,0.0,0.071265
